In [1]:
import os
import json
import numpy as np
# import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds


from tensorflow.keras import backend as K

In [7]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# tf.compat.v1.disable_eager_execution()

# tf.compat.v1.disable_v2_behavior()


# Monkey patching for compatability v1/v2
tf.gfile = tf.io.gfile

In [3]:
VOCAB_FILE = 'data/cnn-dm/vocab.txt'
BERT_MODEL_URL = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
SEQ_LEN = 512
MAX_EXAMPLE_LEN = 512

BATCH_SIZE = 1
BUFFER_SIZE = 1

In [4]:
from bert.tokenization import FullTokenizer


def create_bert_tokenizer(vocab_file, do_lower_case=True):
    """
    Return a BERT FullTokenizer
    """                
    return FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)


def create_tokenizer_from_hub_module(bert_hub_url, bert_module=None):
    """
    Get the vocab file and casing info from the Hub module.
    """
    if bert_module is None:
        bert_module = hub.Module(bert_hub_url)
        
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)

    with tf.compat.v1.Session() as sess:
        vocab_file, do_lower_case = sess.run([
            tokenization_info["vocab_file"],
            tokenization_info["do_lower_case"]
        ])
                
    return FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)



In [8]:
from functools import partial


def pad(l, n, pad=0):
    """
    Pad the list 'l' to have size 'n' using 'padding_element'
    """
    pad_with = (0, max(0, n - len(l)))
    return np.pad(l, pad_with, mode='constant', constant_values=pad)


def load_cnn_dailymail():
    """
    Load the CNN/DM data from tensorflow Datasets
    """
    examples, metadata = tfds.load('cnn_dailymail', with_info=True, as_supervised=True)
    train, val, test = examples['train'], examples['validation'], examples['test']
    return train, val, test, metadata


def encode(sent_1, sent_2, tokenizer, seq_len):
    """
    Encode the text to the BERT expected format

    BERT has the following special tokens:    
    
    [CLS] : The first token of every sequence. A classification token
    which is normally used in conjunction with a softmax layer for classification
    tasks. For anything else, it can be safely ignored.

    [SEP] : A sequence delimiter token which was used at pre-training for
    sequence-pair tasks (i.e. Next sentence prediction). Must be used when
    sequence pair tasks are required. When a single sequence is used it is just appended at the end.

    [MASK] : Token used for masked words. Only used for pre-training.
    
    Additionally BERT requires additional inputs to work correctly:
        - Mask IDs
        - Segment IDs
    
    The mask has 1 for real tokens and 0 for padding tokens. Only real tokens are attended to.
    Sentence Embeddings is just a numeric class to distinguish between pairs of sentences.
    """
    tokens_1 = tokenizer.tokenize(sent_1.numpy())
    tokens_2 = tokenizer.tokenize(sent_1.numpy())
    
    # Account for [CLS] and [SEP] with "- 2"
    if len(tokens_1) > seq_len - 2:
        tokens_1 = tokens_1[0:(seq_len - 2)]
    if len(tokens_2) > seq_len - 2:
        tokens_2 = tokens_2[0:(seq_len - 2)]        
    
    input_ids_1 = tokenizer.convert_tokens_to_ids(tokens_1)
    input_ids_2 = tokenizer.convert_tokens_to_ids(tokens_2)
    
    input_mask_1 = [1] * len(input_ids_1)
    input_mask_2 = [1] * len(input_ids_2)

    input_ids_1 = pad(input_ids_1, seq_len, 0)
    input_ids_2 = pad(input_ids_2, seq_len, 0)
    input_mask_1 = pad(input_mask_1, seq_len, 0)
    input_mask_2 = pad(input_mask_2, seq_len, 0)
    
    input_type_ids_1 = [0] * len(input_ids_1)
    input_type_ids_2 = [0] * len(input_ids_2)
    
    return input_ids_1, input_mask_1, input_type_ids_1, input_ids_2, input_mask_2, input_type_ids_2


def tf_encode(tokenizer, seq_len):
    """
    Operations inside `.map()` run in graph mode and receive a graph
    tensor that do not have a `numpy` attribute.
    The tokenizer expects a string or Unicode symbol to encode it into integers.
    Hence, you need to run the encoding inside a `tf.py_function`,
    which receives an eager tensor having a numpy attribute that contains the string value.
    """    
    def f(s1, s2):
        encode_ = partial(encode, tokenizer=tokenizer, seq_len=seq_len)
        return tf.py_function(encode_, [s1, s2], [tf.int64, tf.int64, tf.int64, tf.int64, tf.int64, tf.int64])
    
    return f


def filter_max_length(x, x1, x2, y, y1, y2, max_length=MAX_EXAMPLE_LEN):
    predicate = tf.logical_and(
        tf.size(x[0]) <= max_length,
        tf.size(y[0]) <= max_length
    )
    return predicate



train_examples, val_examples, test_examples, metadata = load_cnn_dailymail()   

tokenizer = create_bert_tokenizer(VOCAB_FILE)

train_dataset = train_examples.map(tf_encode(tokenizer, SEQ_LEN))

train_dataset = train_dataset.filter(filter_max_length)

train_dataset = train_dataset.cache()

train_dataset = train_dataset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, padded_shapes=([-1], [-1], [-1], [-1], [-1], [-1]))

train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

AttributeError: module 'tensorflow' has no attribute 'gfile'

In [21]:

a = iterator.get_next()
a

<tf.Tensor 'IteratorGetNext_6:0' shape=(5,) dtype=int32>

In [23]:

# dataset1 = tf.data.Dataset.from_tensors(tf.range(10, 15))

with tf.compat.v1.Session() as session:
    session.run(tf.compat.v1.global_variables_initializer())
    print(session.run(a))
    



NotFoundError: Resource AnonymousIterator/AnonymousIterator5/N10tensorflow4data16IteratorResourceE does not exist.
	 [[node IteratorGetNext_6 (defined at <ipython-input-21-bf027f78267d>:2) ]]

Errors may have originated from an input operation.
Input Source operations connected to node IteratorGetNext_6:
 AnonymousIteratorV2_3 (defined at <ipython-input-17-8efe1dbf1309>:1)

Original stack trace for 'IteratorGetNext_6':
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.6/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/usr/local/lib/pytho

In [17]:
iterator = iter(dataset1)


In [12]:
with tf.compat.v1.Session() as session:
    session.run(tf.compat.v1.global_variables_initializer())
    for entry in train_dataset.take(1):
        print(entry)
        print(session.run(entry))
    

(<tf.Tensor 'IteratorGetNext_2:0' shape=(?, ?) dtype=int64>, <tf.Tensor 'IteratorGetNext_2:1' shape=(?, ?) dtype=int64>, <tf.Tensor 'IteratorGetNext_2:2' shape=(?, ?) dtype=int64>, <tf.Tensor 'IteratorGetNext_2:3' shape=(?, ?) dtype=int64>, <tf.Tensor 'IteratorGetNext_2:4' shape=(?, ?) dtype=int64>, <tf.Tensor 'IteratorGetNext_2:5' shape=(?, ?) dtype=int64>)


NotFoundError: Resource AnonymousIterator/AnonymousIterator2/N10tensorflow4data16IteratorResourceE does not exist.
	 [[node IteratorGetNext_2 (defined at <ipython-input-12-47e3038298ef>:3) ]]

Original stack trace for 'IteratorGetNext_2':
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.6/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.6/dist-packages/tornado/platform/asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.6/asyncio/base_events.py", line 438, in run_forever
   

In [ ]:

with tf.compat.v1.Session() as session:
    sess.run(tf.global_variables_initializer())

    model.predict((x0,x1,x2))

In [7]:
for entry in tfds.as_numpy(train_dataset.take(2)):
    x0, x1, x2, y0, y1, y2 = entry
    for i in entry:
        print(i.shape)
    print()
    
    
#     a, b, c = x
#     print(a.shape)
#     print(a)
#     print()
#     print(b.shape)
#     print(b)
#     print()    

#     print(c.shape)
#     print(c)
#     print()
 

W0523 12:24:09.509178 139861868648192 backprop.py:820] The dtype of the watched tensor must be floating (e.g. tf.float32), got tf.string
W0523 12:24:09.786260 139861868648192 backprop.py:820] The dtype of the watched tensor must be floating (e.g. tf.float32), got tf.string
W0523 12:24:09.907563 139861893826304 backprop.py:820] The dtype of the watched tensor must be floating (e.g. tf.float32), got tf.string
W0523 12:24:09.914924 139861893826304 backprop.py:820] The dtype of the watched tensor must be floating (e.g. tf.float32), got tf.string
W0523 12:24:10.082614 139861885433600 backprop.py:820] The dtype of the watched tensor must be floating (e.g. tf.float32), got tf.string


(1, 512)
(1, 512)
(1, 512)
(1, 512)
(1, 512)
(1, 512)

(1, 512)
(1, 512)
(1, 512)
(1, 512)
(1, 512)
(1, 512)



In [ ]:
print(model((x0, x1, x2)))


In [8]:
class BertLayer(tf.keras.layers.Layer):
    """
    Custom Keras layer, integrating BERT from tf-hub
    """
    def __init__(self, url, seq_len=512, d_embedding=768, n_fine_tune_layers=None, **kwargs):
        self.url = url
        self.n_fine_tune_layers = n_fine_tune_layers
        self.seq_len = seq_len
        self.d_embedding = d_embedding
        super(BertLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.bert = hub.Module(
            self.url,
            trainable=self.trainable,
            name="{}_bert_module".format(self.name)
        )

        trainable_vars = self.bert.variables
        
        if self.trainable:

            # Remove unused layers
            trainable_vars = [var for var in trainable_vars if not "/cls/" in var.name]

            # Select how many layers to fine tune
            trainable_vars = trainable_vars[-self.n_fine_tune_layers :]

            # Add to trainable weights
            for var in trainable_vars:
                self._trainable_weights.append(var)

            for var in self.bert.variables:
                if var not in self._trainable_weights:
                    self._non_trainable_weights.append(var)

        super(BertLayer, self).build(input_shape)

    def call(self, inputs):
        inputs = [K.cast(x, dtype="int32") for x in inputs]
        
        input_ids, input_mask, segment_ids = inputs
        
        bert_inputs = dict(input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids)
        result = self.bert(inputs=bert_inputs, signature="tokens", as_dict=True)["sequence_output"]
        return result

    def compute_output_shape(self, input_shape):
        raise ValueError()
        return (input_shape[0], self.seq_len, self.d_embedding)


In [9]:
in_id = tf.keras.layers.Input(shape=(SEQ_LEN,), name="input_ids")
in_mask = tf.keras.layers.Input(shape=(SEQ_LEN,), name="input_masks")
in_segment = tf.keras.layers.Input(shape=(SEQ_LEN,), name="segment_ids")

bert_inputs = [in_id, in_mask, in_segment]

bert = BertLayer(BERT_MODEL_URL, trainable=False)(bert_inputs)

model = tf.keras.models.Model(inputs=bert_inputs, outputs=bert)


In [11]:
model

In [29]:
N = 8
A = np.array([1, 2, 3, 4, 5])
np.pad(A, (0, max(0, N - len(A))), mode='constant')

array([1, 2, 3, 4, 5, 0, 0, 0])

In [21]:
examples, metadata = tfds.load('ted_hrlr_translate/pt_to_en', with_info=True,
                               as_supervised=True)
train_examples, val_examples = examples['train'], examples['validation']

tokenizer_en = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    (en.numpy() for pt, en in train_examples), target_vocab_size=2**13)

tokenizer_pt = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    (pt.numpy() for pt, en in train_examples), target_vocab_size=2**13)




for x,y in tfds.as_numpy(train_dataset.take(11)):
    print(x.shape)

AttributeError: 'Tensor' object has no attribute 'numpy'

In [8]:
with tf.compat.v1.Session() as session:
    
    for x, y in train.take(2):
        print(session.run(x))
        break


NotFoundError: Resource AnonymousIterator/AnonymousIterator0/N10tensorflow4data16IteratorResourceE does not exist.
	 [[node IteratorGetNext_4246 (defined at <ipython-input-8-b27e89ea609b>:3) ]]

Original stack trace for 'IteratorGetNext_4246':
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.6/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.6/dist-packages/tornado/platform/asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.6/asyncio/base_events.py", line 438, in run_foreve

In [ ]:
for (batch, (inp, tar)) in enumerate(train_dataset):
